In [1]:
suppressPackageStartupMessages(library(readxl))
suppressPackageStartupMessages(library(dplyr))
suppressPackageStartupMessages(library(tidyr))
suppressPackageStartupMessages(library(tibble))
suppressPackageStartupMessages(library(ggplot2))
suppressPackageStartupMessages(library(patchwork))
suppressPackageStartupMessages(library(Seurat))
suppressPackageStartupMessages(library(SeuratDisk))
suppressPackageStartupMessages(library(stringr))
library(hise)
library(plyr)
library(purrr)
suppressPackageStartupMessages(library(H5weaver))
library(parallel)

------------------------------------------------------------------------------

You have loaded plyr after dplyr - this is likely to cause problems.
If you need functions from both plyr and dplyr, please load plyr first, then dplyr:
library(plyr); library(dplyr)

------------------------------------------------------------------------------


Attaching package: ‘plyr’


The following objects are masked from ‘package:dplyr’:

    arrange, count, desc, failwith, id, mutate, rename, summarise,
    summarize



Attaching package: ‘purrr’


The following object is masked from ‘package:plyr’:

    compact




In [2]:
reference <- readRDS("/home//jupyter/pbmc_multimodal_2023.rds")
#add level 2.5 labels
l3 <- as.character(reference[[]]$celltype.l3)
l2 <- as.character(reference[[]]$celltype.l2)
l2.5 <- l2
l2.5[l3 == "Treg Naive"] <- "Treg Naive"
l2.5[l3 == "Treg Memory"] <- "Treg Memory"
l2.5[l3 %in% c("CD8 TEM_4", "CD8 TEM_5")] <- "CD8 TEMRA"
reference <- AddMetaData(reference, metadata = l2.5, col.name = "celltype.l2.5")

Loading required package: SeuratObject

Loading required package: sp


Attaching package: ‘SeuratObject’


The following object is masked from ‘package:base’:

    intersect


Loading required package: Seurat



In [9]:
sort(rownames(reference@assays$ADT@data))

[1] "B7-H4"         "C5L2"          "Cadherin"      "CCR10"        
  [5] "CD102"         "CD103"         "CD105"         "CD106"        
  [9] "CD107a"        "CD109"         "CD110"         "CD112"        
 [13] "CD115"         "CD117"         "CD119"         "CD11a/CD18"   
 [17] "CD11b-1"       "CD11b-2"       "CD11c"         "CD122"        
 [21] "CD123"         "CD124"         "CD126"         "CD127"        
 [25] "CD13"          "CD133-1"       "CD133-2"       "CD134"        
 [29] "CD135"         "CD137"         "CD138-1"       "CD138-2"      
 [33] "CD14"          "CD140a"        "CD140b"        "CD141"        
 [37] "CD142"         "CD144"         "CD146"         "CD15"         
 [41] "CD152"         "CD154"         "CD155"         "CD158"        
 [45] "CD158b"        "CD158e1"       "CD158f"        "CD16"         
 [49] "CD161"         "CD163"         "CD164"         "CD169"        
 [53] "CD171"         "CD172a"        "CD177"         "CD178"        
 [57] "CD18"          "CD184"         "CD185"         "CD186"        
 [61] "CD19"          "CD192"         "CD193"         "CD194"        
 [65] "CD195"         "CD196"         "CD199"         "CD1a"         
 [69] "CD1c"          "CD1d"          "CD2"           "CD20"         
 [73] "CD200"         "CD201"         "CD202b"        "CD203c"       
 [77] "CD204"         "CD205"         "CD206"         "CD207"        
 [81] "CD209"         "CD21"          "CD22"          "CD223"        
 [85] "CD226"         "CD235a"        "CD235ab"       "CD24"         
 [89] "CD243"         "CD244"         "CD25"          "CD252"        
 [93] "CD253"         "CD26-1"        "CD26-2"        "CD267"        
 [97] "CD268"         "CD269"         "CD27"          "CD270"        
[101] "CD271"         "CD272"         "CD273"         "CD274"        
[105] "CD275-1"       "CD275-2"       "CD278"         "CD279"        
[109] "CD28"          "CD284"         "CD29"          "CD294"        
[113] "CD3-1"         "CD3-2"         "CD30"          "CD301"        
[117] "CD303"         "CD304"         "CD305"         "CD307c/FcRL3" 
[121] "CD307d"        "CD307e"        "CD309"         "CD31"         
[125] "CD314"         "CD319"         "CD324"         "CD325"        
[129] "CD335"         "CD337"         "CD338"         "CD34"         
[133] "CD340"         "CD35"          "CD354"         "CD357"        
[137] "CD36"          "CD366"         "CD370"         "CD38-1"       
[141] "CD38-2"        "CD39"          "CD4-1"         "CD4-2"        
[145] "CD40"          "CD41"          "CD42b"         "CD43"         
[149] "CD44-1"        "CD44-2"        "CD45-1"        "CD45-2"       
[153] "CD45RA"        "CD45RB"        "CD45RO"        "CD46"         
[157] "CD47"          "CD48"          "CD49a"         "CD49b"        
[161] "CD49d"         "CD52"          "CD54"          "CD55"         
[165] "CD56-1"        "CD56-2"        "CD57"          "CD59"         
[169] "CD61"          "CD62E"         "CD62P"         "CD63"         
[173] "CD64"          "CD66a/c/e"     "CD66b"         "CD68"         
[177] "CD69"          "CD70"          "CD71"          "CD72"         
[181] "CD73"          "CD79a"         "CD79b"         "CD8"          
[185] "CD80"          "CD81"          "CD83"          "CD85g"        
[189] "CD86"          "CD8a"          "CD9"           "CD90"         
[193] "CD93"          "CD95"          "CD96"          "CD98"         
[197] "CD99"          "CLEC12A"       "CLEC2"         "CX3CR1"       
[201] "Folate"        "Galectin-9"    "GP130"         "HLA-DR"       
[205] "IgD"           "IgM"           "Integrin-7"    "LOX-1"        
[209] "MERTK"         "Notch-1"       "Notch-2"       "Podoplanin"   
[213] "Rag-IgG2c"     "Rat-IgG1-1"    "Rat-IgG1-2"    "Rat-IgG2b"    
[217] "Siglec-8"      "TCR-1"         "TCR-2"         "TCR-V-2"      
[221] "TCR-V-24-J-18" "TCR-V-7.2"     "TCR-V-9"       "TIGIT"        
[225] "TIM-4"         "TSLPR"         "VEGFR-3"       "XCR1"

In [4]:
hise_meta<-read.csv("hise_meta_data_2023-11-19.csv")

In [5]:
get_last_pattern <- function(x) {
  split_vector <- strsplit(x, "/")[[1]]
  last_pattern <- tail(split_vector, 1)
  return(last_pattern)
}
last_patterns <- unlist(lapply(hise_meta$file.name, get_last_pattern))


In [6]:
#hise_meta$file.path<-paste0("cache/",hise_meta$file.id,"/",last_patterns)

In [7]:
hise_meta <-hise_meta %>% arrange(file.batchID)

In [8]:
b <- seq(10, dim(hise_meta)[1]+8, 10)
df_chunk_list<-lapply(seq_along(b), function(i) hise_meta[(b-9)[i]:b[i], ])

In [9]:
length(df_chunk_list)

[1] 11

In [ ]:
mclapply(df_chunk_list,function(x){
so_list <- lapply(x$file.path[!is.na(x$file.path)],function(i){
                sce <- read_h5_sce(i)
                counts <- assay(sce, "counts")
                rownames(counts)<- make.names(rownames(counts), unique=TRUE)
                so<-CreateSeuratObject(
                  counts,meta.data =data.frame(sce@colData),
                  assay = "RNA")
                })
combined <- Reduce(merge, so_list)
rm(so_list)
combined <- SCTransform(combined,method = "glmGamPoi", verbose = FALSE)

#find anchors
anchors <- FindTransferAnchors(
  reference = reference,
  query = combined,
  normalization.method = "SCT",
  reference.reduction = "spca",
  dims = 1:50
)  
    
#perform projection to get labels
combined <- MapQuery(
  anchorset = anchors,
  query = combined,
  reference = reference,
  refdata = list(
    celltype.l1 = "celltype.l1",
    celltype.l2 = "celltype.l2",
      celltype.l3 = "celltype.l3",
      celltype.l2.5 = "celltype.l2.5",
    predicted_ADT = "ADT"
  ),
  reference.reduction = "spca", 
  reduction.model = "wnn.umap"
)

mclapply( unique(combined[[]]$pbmc_sample_id),function(i){

labels<-combined[[]]%>% filter(pbmc_sample_id==i)
write.csv(labels,paste0("Labels_20231119/",i,".csv"))
    
combined_sub<-subset(combined,subset=pbmc_sample_id==i)
mat<-combined_sub@assays$predicted_ADT@data
list_mat <- list(i = mat@i, p = mat@p, x = mat@x, Dim = dim(mat),rownames = rownames(mat),
  colnames = colnames(mat))

h5createFile(paste0("Labels_20231119/",i,"_ADT.h5"))

h5write(list_mat, paste0("Labels_20231119/",i,"_ADT.h5"), "mat")
},mc.cores=n_distinct(combined[[]]$pbmc_sample_id))
    
rm(combined)
gc()
},mc.cores=3)